In [1]:
!pip3 install git+git://github.com/geopandas/geopandas.git
!pip3 install descartes
!pip3 install folium
!pip3 install geopy

  Cloning git://github.com/geopandas/geopandas.git to /tmp/pip-req-build-ey3g0ng8
  Running command git clone -q git://github.com/geopandas/geopandas.git /tmp/pip-req-build-ey3g0ng8
     |████████████████████████████████| 14.7MB 308kB/s 
     |████████████████████████████████| 10.9MB 167kB/s 
  Created wheel for geopandas: filename=geopandas-0.8.0+5.g89908f8-py2.py3-none-any.whl size=963135 sha256=fd56e46a82650fbee3e4dc603033d7a077402ff882a534ad926b101d25274f05
  Stored in directory: /tmp/pip-ephem-wheel-cache-qmxowxbt/wheels/91/24/71/376c9c67192694168352afcccc2d264248f7e2cc6192997186
Successfully built geopandas


In [5]:
import pandas as pd
import geopandas as gpd 
import matplotlib.pyplot as plt
import folium
from geopy.geocoders import Nominatim

In [11]:
#Example

locator = Nominatim(user_agent='myGeocoder')
location = locator.geocode('S-20-151, Fairfield, SC')
print('Latitude = {}, Longitude = {}'.format(location.latitude, location.longitude))

Latitude = 40.76892042845975, Longitude = -111.93893027676306


In [8]:
#df = pd.read_csv('/data/Fairfield_South_Carolina_Properties.csv', error_bad_lines=False)
df = pd.read_csv('properties.csv', error_bad_lines=False)
df['Address'] = df['Address']+', '+df['City']+', '+df['State']

In [18]:
df.head()

,Unnamed: 0,ParcelId,OwnerName,Address,City,State,Zip
0,0,216-00-00-005-000,CURRENT RESIDENT,"2100FT W OF INT S-20-323 & S-20-324, Fairfield...",Fairfield,SC,NaN
1,1,215-00-00-074-000,CURRENT RESIDENT,"293 EMERALD DR TRACT, Fairfield, SC",Fairfield,SC,NaN
2,2,215-00-00-102-000,CURRENT RESIDENT,"285 EMERALD DR, Fairfield, SC",Fairfield,SC,NaN
3,3,026-04-02-005-000,CURRENT RESIDENT,"SW OF GIBSON END RD RD, Fairfield, SC",Fairfield,SC,NaN
4,4,026-04-02-007-000,CURRENT RESIDENT,"557 GIBSON END ROAD, Fairfield, SC",Fairfield,SC,NaN


In [19]:
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df['location'] = df['Address'].apply(geocode)
# 3 - create longitude, latitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('PARCELS B & D-LONGTOWN RD RD, Fairfield, SC',), **{}).
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/geopy/geocoders/base.py", line 344, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "/usr/lib/python3.6/urllib/request.py", line 526, in open
    response = self._open(req, data)
  File "/usr/lib/python3.6/urllib/request.py", line 544, in _open
    '_open', req)
  File "/usr/lib/python3.6/urllib/request.py", line 504, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.6/urllib/request.py", line 1368, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "/usr/lib/python3.6/urllib/request.py", line 1328, in do_open
    r = h.getresponse()
  File "/usr/lib/python3.6/http/client.py", line 1356, in getresponse
    response.begin()
  File "/usr/lib/python3.6/http/client.py", line 307, in begin
    version, status, re

In [20]:
df

,Unnamed: 0,ParcelId,OwnerName,Address,City,State,Zip,location,point
0,0,216-00-00-005-000,CURRENT RESIDENT,"2100FT W OF INT S-20-323 & S-20-324, Fairfield...",Fairfield,SC,NaN,None,None
1,1,215-00-00-074-000,CURRENT RESIDENT,"293 EMERALD DR TRACT, Fairfield, SC",Fairfield,SC,NaN,None,None
2,2,215-00-00-102-000,CURRENT RESIDENT,"285 EMERALD DR, Fairfield, SC",Fairfield,SC,NaN,"(285, Emerald Drive, Fairfield County, South C...","(34.230154178930135, -81.07613742999173, 0.0)"
3,3,026-04-02-005-000,CURRENT RESIDENT,"SW OF GIBSON END RD RD, Fairfield, SC",Fairfield,SC,NaN,None,None
4,4,026-04-02-007-000,CURRENT RESIDENT,"557 GIBSON END ROAD, Fairfield, SC",Fairfield,SC,NaN,"(Gibson End Road, Mitford, Fairfield County, S...","(34.529672, -80.911314, 0.0)"
...,...,...,...,...,...,...,...,...,...
17239,17480,141-00-00-031-000,CURRENT RESIDENT,"S 2000FT INT S-20-115 & HWY 34 E H, Fairfield, SC",Fairfield,SC,NaN,None,None
17240,17481,077-01-00-002-000,CURRENT RESIDENT,"1704 GREAT NORTH RD RD, Fairfield, SC",Fairfield,SC,NaN,"(1704, Great North Road, Bucklick, Fairfield C...","(34.45738830853113, -80.92219931688999, 0.0)"
17241,17482,048-00-00-002-000,CURRENT RESIDENT,"HWY 215 N, CLOWNEY & JOHN BRICE RDS, Fairfield...",Fairfield,SC,NaN,None,None
17242,17483,047-00-00-047-000,CURRENT RESIDENT,"JOHN BRICE RD, Fairfield, SC",Fairfield,SC,NaN,"(John Brice Road, Feasterville, Fairfield Coun...","(34.482181, -81.354336, 0.0)"


In [21]:
df.to_csv('latitudes_longitudes.csv', index=False)